In [15]:
import requests, time, pandas as pd
import concurrent.futures, threading

In [16]:
thread_local = threading.local()

In [3]:
def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

In [4]:
def download_df(year):
    base_url = "/".join([HOST, str(year), dataset])
    r = requests.get(base_url, params=predicates)
    df = pd.DataFrame(columns=r.json()[0], data=r.json()[1:])
    df["year"] = year
    return df

In [5]:
def download_all_dfs(years):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        dfs = executor.map(download_df, years)
    return list(dfs)

In [9]:
HOST, dataset = "https://api.census.gov/data", "acs/acs1"
get_vars = ["B25045_" + str(i + 1).zfill(3) + "E" for i in range(19)]
get_vars = ["NAME"] + get_vars

In [10]:
predicates = {}
predicates["get"] = ",".join(get_vars)
predicates["for"] = "state:41"

In [12]:
start_time = time.time()
years = range(2012, 2018)
dfs = download_all_dfs(years)
duration = time.time() - start_time
print(f"Duration: {duration} seconds")

Duration: 3.00396990776062 seconds


In [14]:
pd.concat(dfs)

,NAME,B25045_001E,B25045_002E,B25045_003E,B25045_004E,B25045_005E,B25045_006E,B25045_007E,B25045_008E,B25045_009E,...,B25045_012E,B25045_013E,B25045_014E,B25045_015E,B25045_016E,B25045_017E,B25045_018E,B25045_019E,state,year
0,Oregon,1516957,933775,24706,1616,10168,12922,909069,81070,560911,...,100722,29609,40339,30774,482460,196749,232007,53704,41,2012
0,Oregon,1523799,926930,21385,1169,9729,10487,905545,70865,556199,...,100596,26475,41074,33047,496273,195479,249400,51394,41,2013
0,Oregon,1535511,932392,22969,1261,9166,12542,909423,75489,547692,...,97641,25282,38944,33415,505478,197304,247609,60565,41,2014
0,Oregon,1553205,948891,22884,1238,11026,10620,926007,81009,541331,...,104229,26261,47188,30780,500085,185900,251830,62355,41,2015
0,Oregon,1571678,969579,22501,1712,7635,13154,947078,85622,553387,...,93257,22436,34407,36414,508842,191455,250413,66974,41,2016
0,Oregon,1603635,1007015,22530,1383,9594,11553,984485,95057,559321,...,93586,24106,38751,30729,503034,186749,253297,62988,41,2017
